# VI. Create spend calculations

Alice has 69300000 piconeros to spend. But to spend the piconeros she will have to pay a transaction fee, and will have to subtract this transaction fee from the total amount she can send. To obfuscate spends, Monero always requires at least two outputs. In this case the second output will be a 0 amount output sent to a random address.

There are two primary goal for this section:
1. generate transaction data Bob will use to identify Monero destined to him
2. generate transaction data for the left over Monero Alice sends back to herself (it will be zero), this step will be explained

This section relies heavliy on the following source:
* [Zero to Monero: Second Edition; Chapter 5](https://www.getmonero.org/library/Zero-to-Monero-2-0-0.pdf)

**It is highly recommended that the reader read chapter 4 and chapter 5 of _Zero to Monero: Second Edition_ prior to reading the following sections.**

The sections below cover:
1. determining the transaction fee
2. encrypting amounts
3. calculating blinding factors
4. calculating commitments
5. calculating the pseudo-commitment

### Summary (Introduction)
**What we start with:**
* Alice's public view key: `b981d77369cbf23725d61cceb6f6686b7e435ca52e2ccf518f560cc71fc54867`
* Alice's public spend key: `cac88399eed832989df44ae91a0df5650040be8cfbe21b9570466432e6286d37`
* Bob's public spend key: `c95c75de8c46a69ec8b190ce1fde47165a686385910fafc69d2472ffeaddcfc3`
* Bob's public view key: `2d67eaa5f16c0c46e3052dccd0dc036f3d35bca0855f5af57e9997b63f6506bb`
* transaction key (r): `42adc98ec7a15451f3a21750b2172e14d851191a7c07d247591bbd063b698a02`
* Monero to be spent (piconero): `69300000`

**What we end up with:**
* transaction fee
* decrypted spend amount (one for Bob, change is zero)
* encrypted spend amount (one for Bob, one for change)
* commitment blinding factors (one for Bob, one for change)
* commitments (one for Bob, one for change)
* pseudo-commitment
* sum of blinding factors

## 1. Determining the transaction fee

The transaction fee is normally calculated by a wallet. Here it is simply retreived from the actual transaction spend.

In [1]:
# read data directly from the saved transaction
import json

with open("../../transactions/outbound/txn_9e29.json", "r") as file:
    txn = json.load(file)

# extract transaction fee from transaction data
transaction_fee = txn["rct_signatures"]["txnFee"]


print(f"Transaction fee: {transaction_fee}")

Transaction fee: 30720000


## 2. Encrypting amounts

This section essentially reverses the steps Alice previously performed when she identified an amount addressed to her.

In [2]:
'''
Zero to Monero: Second Edition; Section 5.3 (not including the commitment part)
'''


'''
To encrypt the payment amount this code performs the following:

  1. Uses HrKv as a bit mask
  2. Calculates the amount Alice will send after subtracting transaction fees
  2. Applies the mask to the amount field
'''

import nacl.bindings
import varint

from Cryptodome.Hash import keccak
from binascii import hexlify, unhexlify


# define a function to return a hashed byte string
def keccak_256(data):
    return keccak.new(digest_bits=256).update(data).digest()

# define a function to convert an integer into a valid ed25519 scalar
def pad_and_reduce(scalar: bytes) -> bytes:
    point_padded_to_64_bytes = scalar + (64 - len(scalar)) * b"\0"
    return nacl.bindings.crypto_core_ed25519_scalar_reduce(point_padded_to_64_bytes)

# define a function to calculate exclusive or between bits in two variables
def calculate_xor(var1, var2):
    return bytearray(a ^ b for a, b in zip(*map(bytearray, (var1, var2))))


### begin section: this is repeated from the view tag section for clarity
r = "42adc98ec7a15451f3a21750b2172e14d851191a7c07d247591bbd063b698a02"
public_view_key_bob = "2d67eaa5f16c0c46e3052dccd0dc036f3d35bca0855f5af57e9997b63f6506bb"
scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp
cofactor = int(8).to_bytes(32, byteorder="little")
rKv = scalar_point_mult(unhexlify(r), unhexlify(public_view_key_bob))
rKv = scalar_point_mult(cofactor, rKv)
rKv_concat_extra_index = rKv + varint.encode(1)
HrKv = keccak_256(rKv_concat_extra_index)
### end section


# ensure the hash is a valid scalar
amount_mask = pad_and_reduce(HrKv)

# calculate Keccak hash of concatenated "amount" and scalar
amount_mask = keccak_256(b"amount" + amount_mask)

# alice owns this amount of Monero
alice_owned_amount = 69300000

# alice will send this amount to Bob
amount_1 = alice_owned_amount - transaction_fee

# convert amount to 8 bytes
amount_1 = int(amount_1).to_bytes(8, byteorder="little")

# truncate obfuscation mask to length of amount field
truncated_amount_mask = amount_mask[: len(amount_1)]

# apply binary xor operation to encrypted amount
encrypted_amount_1 = calculate_xor(amount_1, truncated_amount_mask)


print(f"Amount for Bob (Output 1): {int.from_bytes(amount_1, byteorder='little')}")
print(f"Encrypted amount for Bob (Output 1): {hexlify(encrypted_amount_1).decode()}")

Amount for Bob (Output 1): 38580000
Encrypted amount for Bob (Output 1): d878feb0045449ee


In [3]:
'''
Zero to Monero: Second Edition; Section 5.3 (not including the commitment part)
'''


'''
To encrypt the payment amount this code performs the following:

  1. Uses HrKv as a bit mask
  2. Calculates the amount Alice will send after subtracting transaction fees
  2. Applies the mask to the amount field
'''

### begin section: this is repeated from the view tag section for clarity
public_view_key_alice = "b981d77369cbf23725d61cceb6f6686b7e435ca52e2ccf518f560cc71fc54867"
scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp
cofactor = int(8).to_bytes(32, byteorder="little")
rKv = scalar_point_mult(unhexlify(r), unhexlify(public_view_key_alice))
rKv = scalar_point_mult(cofactor, rKv)
rKv_concat_extra_index = rKv + varint.encode(0)
HrKv = keccak_256(rKv_concat_extra_index)
### end section


# ensure the hash is a valid scalar
amount_mask = pad_and_reduce(HrKv)

# calculate Keccak hash of concatenated "amount" and scalar
amount_mask = keccak_256(b"amount" + amount_mask)

# zero amount
amount_0 = int(0).to_bytes(8, byteorder="little")

# truncate obfuscation mask to length of amount field
truncated_amount_mask = amount_mask[: len(amount_0)]

# apply binary xor operation to encrypted amount
encrypted_amount_0 = calculate_xor(amount_0, truncated_amount_mask)


print(f"Amount for zero change (Output 0): {0}")
print(f"Encrypted amount for zero change (Output 0): {hexlify(encrypted_amount_0).decode()}")

Amount for zero change (Output 0): 0
Encrypted amount for zero change (Output 0): d9311c21fa3168e1


## 3. Calculating blinding factors

Alice will used the blinding factors (y) she creates here to later create a pseudo-commitment.

In [4]:
'''
Zero to Monero: Second Edition; Section 5.3
'''


'''
To decode the commitment field this code performs the following:

  1. Concatenates the byte string "commitment_mask" and the value HrKv
  2. Ensures this is a valid curve scalar and stores it as "y" - the blinding factor
'''

import nacl.bindings

from binascii import hexlify, unhexlify


def pad_and_reduce(point):
    point_padded_to_64_bytes = point + (64 - len(point)) * b"\0"
    return nacl.bindings.crypto_core_ed25519_scalar_reduce(point_padded_to_64_bytes)


### begin section: this is repeated from 2_ownership_calculation for clarity
scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp
cofactor = int(8).to_bytes(32, byteorder="little")
rKv = scalar_point_mult(unhexlify(r), unhexlify(public_view_key_bob))
rKv = scalar_point_mult(cofactor, rKv)
rKv_concat_extra_index = rKv + varint.encode(1)
HrKv = keccak_256(rKv_concat_extra_index)
HrKv = pad_and_reduce(HrKv)
### end section


#  y = hash("commitment_mask" + hash(rKv + index))
y_1 = keccak_256(b"commitment_mask" + HrKv)

# ensure y is a valid ed25519 scalar
y_1 = pad_and_reduce(y_1)


print(f"Blinding factor for Bob (Output 1) (y): {hexlify(y_1).decode()}")

Blinding factor for Bob (Output 1) (y): 7ec60a17a63b0fe2e3f4b68d86739f9c562b21ae58f7575b7219f8ff061d130b


In [5]:
'''
Zero to Monero: Second Edition; Section 5.3
'''


'''
To decode the commitment field this code performs the following:

  1. Concatenates the byte string "commitment_mask" and the value HrKv
  2. Ensures this is a valid curve scalar and stores it as "y" - the blinding factor
'''

import nacl.bindings

from binascii import hexlify, unhexlify


def pad_and_reduce(point):
    point_padded_to_64_bytes = point + (64 - len(point)) * b"\0"
    return nacl.bindings.crypto_core_ed25519_scalar_reduce(point_padded_to_64_bytes)


### begin section: this is repeated from 2_ownership_calculation for clarity
scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp
cofactor = int(8).to_bytes(32, byteorder="little")
rKv = scalar_point_mult(unhexlify(r), unhexlify(public_view_key_alice))
rKv = scalar_point_mult(cofactor, rKv)
rKv_concat_extra_index = rKv + varint.encode(0)
HrKv = keccak_256(rKv_concat_extra_index)
HrKv = pad_and_reduce(HrKv)
### end section


#  y = hash("commitment_mask" + hash(rKv + index))
y_0 = keccak_256(b"commitment_mask" + HrKv)

# ensure y is a valid ed25519 scalar
y_0 = pad_and_reduce(y_0)


print(f"Blinding factor for zero change (Output 0) (y): {hexlify(y_0).decode()}")

Blinding factor for zero change (Output 0) (y): ac676979b5bba2bf8f8692f3424b9c37c18cb1ffae4422cd6d871b4ecd51970d


## 4. Calculating commitments

Commitments are a way Alice can communicate hidden amounts in a transaction while still allowing addition and subtraction to be used. The `amount` field is an encrypted amount but addition and subtraction are not meaningful for that field. A commitment can be added and subtracted from other commitments which means the amounts contained within can be added and subtracted while obfuscating the amount.

In [6]:
'''
Zero to Monero: Second Edition; Section 5.3
'''


'''
To recreate the commitment field this code performs the following:

  1. Calculate yG by multiplying the blinding factor (y) by the generator point G
  2. Create the curve point H from the generator point G
  3. Encode the amount (b) as bytes
  4. Calculate bH by multiplying the amount (b) by the point H
  5. Add yG and bH 
'''

# define a function to multiply a scalar by the base point G
scalar_mult_G = nacl.bindings.crypto_scalarmult_ed25519_base_noclamp

# define a function to add to curve points
point_add = nacl.bindings.crypto_core_ed25519_add

# define a function that allows multiplying a curve point by a scalar (eg: integer)
scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp

# Monero multiplies curve points by 8, create 8 in bytes for use in calculations
cofactor = int(8).to_bytes(32, byteorder="little")


# calculate yG
yG = scalar_mult_G(y_1)


### begin section: recreate the curve point H
# obtain the curve point G (uses G*1 = G)
G = scalar_mult_G(int(1).to_bytes(32, byteorder="little"))

# hash G to a number and directly interpret this as the y-coordinate for a curve point
H = keccak_256(G)

# multiply H by the cofactor 8 to ensure it is in the prime order subgroup
H = scalar_point_mult(cofactor, H)
### end section: recreate the curve point H


# recode payment amount as 32 bytes
b = int(38580000).to_bytes(32, byteorder="little")

# calculate bH
bH = scalar_point_mult(b, H)

# calculate commitment: yG + bH
committment_0 = point_add(yG, bH)


print(f"Curve point yG: {hexlify(yG).decode()}")
print(f"Curve point bH: {hexlify(bH).decode()}")
print(f"Commitment (recalculated) (Output 0): {hexlify(committment_0).decode()}")

Curve point yG: cbdc2a5eaac2d0228950b6955264b48b195ad0656cc2be5170446ae0dd78f2df
Curve point bH: bdcb504f17e9fa6ff08d7f9415c6c1090b52c1aba8a28f65ada20122c14eef03
Commitment (recalculated) (Output 0): 3921477a4506f7eddde7ff228f05d8026525276dce1880b3d57e540836ccc8d1


In [7]:
'''
Zero to Monero: Second Edition; Section 5.3
'''


'''
To recreate the commitment field this code performs the following:

  1. Calculate yG by multiplying the blinding factor (y) by the generator point G
  2. Create the curve point H from the generator point G
  3. Encode the amount (b) as bytes
  4. Calculate bH by multiplying the amount (b) by the point H
  5. Add yG and bH 
'''

# define a function to multiply a scalar by the base point G
scalar_mult_G = nacl.bindings.crypto_scalarmult_ed25519_base_noclamp

# define a function to add to curve points
point_add = nacl.bindings.crypto_core_ed25519_add

# define a function that allows multiplying a curve point by a scalar (eg: integer)
scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp

# Monero multiplies curve points by 8, create 8 in bytes for use in calculations
cofactor = int(8).to_bytes(32, byteorder="little")


# calculate yG
yG = scalar_mult_G(y_0)

# calculate commitment: yG + 0H = yG
committment_0 = yG


print(f"Curve point yG: {hexlify(yG).decode()}")
print(f"Curve point bH: null because amount is 0")
print(f"Commitment (recalculated) (Output 0): {hexlify(committment_0).decode()}")

Curve point yG: 8d34848196e903ba8f9ff13d57402ee4e845aba06dbd2aee382f20c9ac866157
Curve point bH: null because amount is 0
Commitment (recalculated) (Output 0): 8d34848196e903ba8f9ff13d57402ee4e845aba06dbd2aee382f20c9ac866157


## 5. Calculating pseudo-commitment

When Alice creates a transaction she will require a way to reference her previously received Monero. Rather than directly referencing the prior commitment, she will make a new commitment to the same amount but with a different blinding factor. This new commitment is called a pseudo-commitment and is stored in the new transaction.

**Note: There is one commitment per spent output in a transaction. In this transaction Alice is spending only one input so there is only one corresponding pseudo-commitment. In this simple case the blinding factor used in the one pseudo-commitment is equal to the sum of blinding factors used in new commitments. When there are more than one pseudo-commitment all of them, except the last one, use random blinding factors, and then the blinding factors for the last pseudo-commitment are set equal to the sum of new output blinding factors less the sum of randomly created blinding factors.** 

In [8]:
'''
Zero to Monero: Second Edition; Section 5.4
'''


'''
To create a pseudo-commitment this code performs the following:

  1. Sum the blinding factors previously calculated (y)
  2. Calculate yG by multiplying the blinding factor sum (y) by the generator point G
  3. Create the curve point H from the generator point G
  4. Sum the amounts and transaction fees 
  5. Encode the amount (b) as bytes
  6. Calculate bH by multiplying the amount (b) by the point H
  7. Add yG and bH 
'''

scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp
point_add = nacl.bindings.crypto_core_ed25519_add
scalar_add = nacl.bindings.crypto_core_ed25519_scalar_add

cofactor = int(8).to_bytes(32, byteorder="little")
G = scalar_mult_G(int(1).to_bytes(32, byteorder="little"))
H = keccak_256(G)
H = scalar_point_mult(cofactor, H)

y = scalar_add(y_0, y_1)

b = int.from_bytes(amount_0, byteorder="little") + int.from_bytes(amount_1, byteorder="little") + transaction_fee
b = int(b).to_bytes(32, byteorder="little")

yG = scalar_mult_G(y)
bH = scalar_point_mult(b, H)

pseudo_commitment = point_add(yG, bH)

print(f"Pseudo-commitment blinding factor (y): {hexlify(y).decode()}")
print(f"Pseudo-commitment amount (b): {hexlify(b).decode()}")
print(f"Pseudo-commitment (yG + bH): {hexlify(pseudo_commitment).decode()}")


Pseudo-commitment blinding factor (y): 3d5a7e3341949f499dde51deeac45cbf17b8d2ad073c7a28e0a0134ed46eaa08
Pseudo-commitment amount (b): 206f210400000000000000000000000000000000000000000000000000000000
Pseudo-commitment (yG + bH): 57340efcce4b3b7fb210de30439fb2fca501284c73411ce657db8f7f03d232e3


### Summary
**What we start with:**
* Bob's Monero address: `49FgswTBtGdTZQPj7e16qc4jrWY5VuGE2aDoaMDEALknZeTy5vcjkcsCrh4deo231LKcACcjgALFji4c4SYQF5WdNCLvkN3`
* Alice's public view key: `b981d77369cbf23725d61cceb6f6686b7e435ca52e2ccf518f560cc71fc54867`
* Alice's public spend key: `cac88399eed832989df44ae91a0df5650040be8cfbe21b9570466432e6286d37`
* Monero to be spend (piconero): `69300000`

**What we end up with:**
* transaction fee: `30720000`
* decrypted spend amount (for Bob): `38580000`
* encrypted spend amount (for Bob): `d878feb0045449ee`
* encrypted zero change amount: `d9311c21fa3168e1`
* blinding factor 1 (for Bob): `7ec60a17a63b0fe2e3f4b68d86739f9c562b21ae58f7575b7219f8ff061d130b`
* blinding factor 0 (for zero change): `ac676979b5bba2bf8f8692f3424b9c37c18cb1ffae4422cd6d871b4ecd51970d`
* commitment 1 (for Bob): `3921477a4506f7eddde7ff228f05d8026525276dce1880b3d57e540836ccc8d1`
* commitment 0 (for zero change): `8d34848196e903ba8f9ff13d57402ee4e845aba06dbd2aee382f20c9ac866157`
* sum of blinding factors: `3d5a7e3341949f499dde51deeac45cbf17b8d2ad073c7a28e0a0134ed46eaa08`
* pseudo-commitment: `57340efcce4b3b7fb210de30439fb2fca501284c73411ce657db8f7f03d232e3`